## Домашнее задание

Для датасета ml-latest реализовать гибридную систему с использованием известных подходов.

In [2]:
from surprise import KNNWithMeans, KNNBasic, SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

from sklearn.model_selection import train_test_split as tts

import pandas as pd

In [3]:
movies = pd.read_csv('MovieLens/movies.csv')
ratings = pd.read_csv('MovieLens/ratings.csv')

К сожалению, на датасете ml-latest умерло ядро (от недостатка оперативной памяти), поэтому был использован старый датасет.

In [4]:
# %%time
# movies = pd.read_csv('ml-latest/movies.csv')
# ratings = pd.read_csv('ml-latest/ratings.csv')

**Подготовка данных**

In [5]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.shape

(100836, 6)

In [6]:
del movies
del ratings

In [7]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

Построим гибридную систему, в основе которой лежит блендинг: в качестве базовых моделей будем использовать *KNNWithMeans* и *KNNBasic*, в качестве метамодели - *SVD* и предскажем рейтинги фильмов. Разделим датасет на тестовую (для метамодели) и обучаемую выборки, а затем обучаемую - также на тестовую и обучаемую для базовых алгоритмов. Обучим базовые модели, а затем сделаем предсказания на тестовых выборках для метамодели и для базовых моделей. На основе полученных результатов сделаем новые обучаемую и тестовую выборки для метамодели, обучим ее и сделаем предсказания. Результат оценим с использованием *RMSE*.

In [8]:
X_train, X_test, y_train, y_test = tts(dataset[['uid', 'iid']], dataset[['rating']], test_size=0.2)

In [9]:
train_data = pd.concat([X_train, y_train], names=['uid', 'iid', 'rating'], axis=1)
test_data = pd.concat([X_test, y_test], names=['uid', 'iid', 'rating'], axis=1)

In [10]:
reader = Reader(rating_scale=(0.5, 5.0))
train = Dataset.load_from_df(train_data, reader)
test = Dataset.load_from_df(test_data, reader)

In [11]:
base_train, holdout_test = train_test_split(train, test_size=0.2)

In [12]:
test = test.build_full_trainset().build_testset()

Обучаем базовые модели

In [13]:
%%time
algo_1 = KNNWithMeans(k=20, sim_options={'name': 'pearson_baseline', 'user_based': False}, verbose=False)
algo_1.fit(base_train)
holdout_pred_1 = algo_1.test(holdout_test)
test_pred_1 = algo_1.test(test)
accuracy.rmse(test_pred_1, verbose=True)

RMSE: 0.9039
Wall time: 29.3 s


0.9039491755051058

In [14]:
%%time
algo_2 = KNNBasic(k=20, sim_options={'name': 'pearson_baseline', 'user_based': False}, verbose=False)
algo_2.fit(base_train)
holdout_pred_2 = algo_2.test(holdout_test)
test_pred_2 = algo_2.test(test)
accuracy.rmse(test_pred_2, verbose=True)

RMSE: 0.9422
Wall time: 26.7 s


0.9422419882005165

Создаем обучаемую и тестовую выборки для метамодели

In [15]:
holdout_test = pd.DataFrame(holdout_test, columns=['uid', 'iid', 'rating'])
holdout_pred_1 = pd.DataFrame(holdout_pred_1)[['uid', 'iid', 'est']]
holdout_pred_2 = pd.DataFrame(holdout_pred_2)[['uid', 'iid', 'est']]
holdout_pred_1.columns = ['uid', 'iid', 'rating']
holdout_pred_2.columns = ['uid', 'iid', 'rating']
data_holdout = pd.concat([holdout_test, holdout_pred_1, holdout_pred_2])

In [16]:
test = pd.DataFrame(test, columns=['uid', 'iid', 'rating'])
test_pred_1 = pd.DataFrame(test_pred_1)[['uid', 'iid', 'est']]
test_pred_2 = pd.DataFrame(test_pred_2)[['uid', 'iid', 'est']]
test_pred_1.columns = ['uid', 'iid', 'rating']
test_pred_2.columns = ['uid', 'iid', 'rating']
data_test = pd.concat([test, test_pred_1, test_pred_2])

In [17]:
data_holdout = Dataset.load_from_df(data_holdout, reader).build_full_trainset()

In [18]:
data_test = Dataset.load_from_df(data_test, reader).build_full_trainset().build_testset()

Обучаем метамодель

In [19]:
%%time
algo_meta = SVD(n_factors=20, n_epochs=20)
algo_meta.fit(data_holdout)

Wall time: 1.87 s


In [20]:
pred_meta = algo_meta.test(data_test)
accuracy.rmse(pred_meta, verbose=True)

RMSE: 0.6385


0.6385089597003689

In [21]:
algo_meta.predict(uid=2.0, iid='Mortal Kombat (1995)')

Prediction(uid=2.0, iid='Mortal Kombat (1995)', r_ui=None, est=3.3900725075812064, details={'was_impossible': False})

Как видим, RMSE получился намного меньше относительно оценок базовых моделей, что говорит о том, что использование блендинга позволяет улучшить качество предсказаний.